# Quantitative sensitivity analysis


## Generalized Sobol Indices


Here we show how to compute generalized Sobol indices on the **EOQ** model using the algorithm presented in Kucherenko et al. 2012. We import our model function from ``temfpy`` and use the Kucherenko indices function from ``econsa``. 

In [1]:
import matplotlib.pyplot as plt  # noqa: F401
import numpy as np

from temfpy.uncertainty_quantification import eoq_model

# TODO: Reactivate once Tim's PR is ready.
# from econsa.kucherenko import kucherenko_indices  # noqa: E265

The function ``kucherenko_indices`` expects the input function to be broadcastable over rows, that is, a row represents the input arguments for one evaluation. For sampling around the mean parameters we specify a diagonal covariance matrix, where the variances depend on the scaling of the mean. Since the variances of the parameters are unknown prior to our analysis we choose values such that the probability of sampling negative values is negligible. We do this since the **EOQ** model is not defined for negative parameters and the normal sampling does not naturally account for bounds.

In [2]:
def eoq_model_transposed(x):
    """EOQ Model but with variables stored in columns."""
    return eoq_model(x.T)


mean = np.array([1230, 0.0135, 2.15])
cov = np.diag([1, 0.000001, 0.01])

# indices = kucherenko_indices( # noqa: E265
#    func=eoq_model_transposed, # noqa: E265
#    sampling_mean=mean,  # noqa: E265
#    sampling_cov=cov,  # noqa: E265
#    n_draws=1_000_000,  # noqa: E265
#    sampling_scheme="sobol",  # noqa: E265
# )  # noqa: E265

Now we are ready to inspect the results.

In [3]:
# sobol_first = indices.loc[(slice(None), "first_order"), "value"].values  # noqa: E265
# sobol_total = indices.loc[(slice(None), "total"), "value"].values # noqa: E265

# x = np.arange(3)  # the label locations  # noqa: E265
# width = 0.35  # the width of the bars  # noqa: E265

# fig, ax = plt.subplots()  # noqa: E265
# rects1 = ax.bar(x - width / 2, sobol_first, width, label="First-order")  # noqa: E265
# rects2 = ax.bar(x + width / 2, sobol_total, width, label="Total")  # noqa: E265

# ax.set_ylim([0, 1])  # noqa: E265
# ax.legend()  # noqa: E265

# ax.set_xticks(x)  # noqa: E265
# ax.set_xticklabels(["$x_0$", "$x_1$", "$x_2$"])  # noqa: E265
# ax.legend();  # noqa: E265

In [4]:
# fig  # noqa: E265

## Shapley value

### Tutorial: A Linear Model with Three Inputs

#### The Execution

In [1]:
# import necessary packages and functions
import numpy as np
import chaospy as cp

from econsa.shapley import get_shapley
from econsa.shapley import _r_condmvn

Load all neccesary inputs for the model, you will need:
- a vector of mean estimates
- a covariance matrix
- the model you are conducting SA on
- the functions ``x_all`` and ``x_cond`` for conditional sampling. These functions depend on the distribution from which you are sampling from - for the purposes of this ilustration, we will sample from a multvariate normal distribution, but the functions can be tailored to needs.

In [4]:
# mean and covaraince matrix inputs
np.random.seed(123)
n_inputs = 3
mean = np.zeros(3)
cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])

In [7]:
# model for which SA is being performed
def gaussian_model(X):
    return np.sum(X,1)

In [8]:
# functions for conditional sampling
def x_all(n):
    distribution = cp.MvNormal(mean, cov)
    return distribution.sample(n)

def x_cond(n, subset_j, subsetj_conditional, xjc):
    if subsetj_conditional is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(subset_j, axis = 1)
        cov_int = cov_int[subset_j]
        distribution = cp.MvNormal(mean[subset_j], cov_int)
        return distribution.sample(n)
    else:
        return _r_condmvn(n, mean = mean, cov = cov, dependent_ind = subset_j, given_ind = subsetj_conditional, x_given = xjc)

In [5]:
# estimate Shapley effects using the exact method
method = 'exact'
n_perms = None
n_output = 10**4
n_outer = 10**3
n_inner = 10**2

get_shapley(method, gaussian_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)

,Shapley effects,std. errors,CI_min,CI_max
X1,0.101309,0.002415,0.096575,0.106044
X2,0.418989,0.162970,0.099568,0.738410
X3,0.479701,0.163071,0.160083,0.799320


In [6]:
# estimate Shapley effects using the random method
method = 'random'
n_perms = 30000
n_output = 10**4
n_outer = 1
n_inner = 3


get_shapley(method, gaussian_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)

,Shapley effects,std. errors,CI_min,CI_max
X1,0.111083,0.002969,0.105263,0.116903
X2,0.415180,0.003141,0.409023,0.421337
X3,0.473737,0.003160,0.467543,0.479931


And there we have the resultant Shapley effects, in line witht he chosen method of estimation.